## STEP 01. Import Modules

In [1]:
import numpy as np
import pandas as pd
import json
from tqdm import *

import numpy as np
import pandas as pd

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt

c:\python38\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
import os
os.getcwd();

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## STEP 02. Set Target csv File List

In [1]:
list = [ 'youtube_comments']

## STEP 03. PREPROCESSING - OPTION

In [4]:
# For user analysis, sometimes spamming and repetitve crawling results are what you need.
# So when you need those results, you can skip this STEP or do it in part.

for name in tqdm(list) :
    name_01 = name + ".csv"    
    df = pd.read_csv(name_01).dropna()
    df.reset_index(drop=True, inplace=True)
       
    print("삭제 대상을 확인 중입니다.")
    
    for i in tqdm(range(len(df))) :
        try :
            target_01 = len( [ x for x in df['title'] if x == df['title'][i] ])
        except :
            target_01 = len( [ x for x in df['cleanBody'] if x == df['cleanBody'][i] ])
            
        target_02 = len( [ y for y in df['url'] if y == df['url'][i] ])
        
        if target_01 > 1 :
            df['title'][i] = "delete target"
        elif target_02 > 1 :
            df['url'][i] = "delete target"
    
    target_list = []
    
    print("삭제 대상을 제거합니다.")
    
    before = len(df)
    
    for j in range(len(df)) :
        if df['title'][j] == "delete target" or df['url'][j] == "delete target" :
            target_list.append(j)
            
    df.drop(df.index[target_list], inplace=True)
    
    after = len(df)
    
    print(f"file name {name} 작업 완료...")
    print(f"총 {before - after}개의 열이 삭제되었습니다.")
    
    if os.path.isdir("./preprocessing") :
        pass
    else :
        os.mkdir("./preprocessing")
    
    df.to_csv(f"./preprocessing/{name_01}", index=False)
    print(f"파일 저장 완료...저장 위치 : ./preprocessing/{name_01}")
    

print("preprocessing 작업을 완료하였습니다.")

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]


삭제 대상을 확인 중입니다.


KeyError: 'url'

## STEP 04. Drawing LDA

In [51]:
df = pd.read_csv("data.csv")
df

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,domain,keyword,date,user_name,user_text
0,twitter,윤석열+x파일,2021-06-17,N24,이재명 같은걸 밀면서윤석열X파일같은 소리 하고 있다.트윗 인용하기허연바람™6월 17...
1,twitter,윤석열+x파일,2021-06-17,조국의시간,윤석열x파일이 어디
2,twitter,윤석열+x파일,2021-06-17,담비,추미애 전 법무부 장관 -대담집 통해 미완의 상태 검찰 개혁 언급할 것 -추-윤 갈...
3,twitter,윤석열+x파일,2021-06-17,시라소니,윤석열X 파일난 떳떳해 충격! 추미애 검찰당 후보 곧 허상 보게된다!https://...
4,twitter,윤석열+x파일,2021-06-17,커리커리,"송영길 ""윤석열X파일, 이명박 BBK처럼 野 경선서 밝혀질 것"" 이 병신봐라 이명..."
...,...,...,...,...,...
1034,twitter,윤석열+x파일,2021-06-23,min,어제 유재일씨가 먼저 알린 그분이구만.. 열린공감tv 열린민주당 관계단체이라던데....
1035,twitter,윤석열+x파일,2021-06-24,이청산,"[폴리뉴스] [유창선 칼럼]윤석열X파일, ‘생태탕 선거’의 연장전 되지 않기를[유창..."
1036,twitter,윤석열+x파일,2021-06-23,국민일보,"장성철 ""'윤석열X파일', 사찰 · 공작 아니다…문건 파기할 것""#SBS뉴스장성철 ..."
1037,twitter,윤석열+x파일,2021-06-24,"TIOT 박원순,가습기살균제 피해자들과 함께 합니다",틀렸습니다. 친여성향이 아니고 친이성향이겠죠. 친여와 친이는 매우 다릅니다. '윤...


In [52]:
df.dropna()

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,domain,keyword,date,user_name,user_text
0,twitter,윤석열+x파일,2021-06-17,N24,이재명 같은걸 밀면서윤석열X파일같은 소리 하고 있다.트윗 인용하기허연바람™6월 17...
1,twitter,윤석열+x파일,2021-06-17,조국의시간,윤석열x파일이 어디
2,twitter,윤석열+x파일,2021-06-17,담비,추미애 전 법무부 장관 -대담집 통해 미완의 상태 검찰 개혁 언급할 것 -추-윤 갈...
3,twitter,윤석열+x파일,2021-06-17,시라소니,윤석열X 파일난 떳떳해 충격! 추미애 검찰당 후보 곧 허상 보게된다!https://...
4,twitter,윤석열+x파일,2021-06-17,커리커리,"송영길 ""윤석열X파일, 이명박 BBK처럼 野 경선서 밝혀질 것"" 이 병신봐라 이명..."
...,...,...,...,...,...
1034,twitter,윤석열+x파일,2021-06-23,min,어제 유재일씨가 먼저 알린 그분이구만.. 열린공감tv 열린민주당 관계단체이라던데....
1035,twitter,윤석열+x파일,2021-06-24,이청산,"[폴리뉴스] [유창선 칼럼]윤석열X파일, ‘생태탕 선거’의 연장전 되지 않기를[유창..."
1036,twitter,윤석열+x파일,2021-06-23,국민일보,"장성철 ""'윤석열X파일', 사찰 · 공작 아니다…문건 파기할 것""#SBS뉴스장성철 ..."
1037,twitter,윤석열+x파일,2021-06-24,"TIOT 박원순,가습기살균제 피해자들과 함께 합니다",틀렸습니다. 친여성향이 아니고 친이성향이겠죠. 친여와 친이는 매우 다릅니다. '윤...


In [16]:
list = ['data']
min_df = 3
max_iteration = 20
NUM_TOPICS = 3

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
df['category'].unique()

In [19]:
for name in list :
    name_01 = name + ".csv"
    
    df = pd.read_csv(name_01).dropna()
    df = df[df['category'] == "international"]
    # Convert to list
    data = df.cleanBody.values.tolist()

    # cleanBody에서 명사만 추출 

    from konlpy.tag import *
    from nltk import Text

    okt = Okt()

    data_morph = []
    for i in range(len(data)):

        try :
            data_morph.append(" ".join(okt.nouns(data[i])))
        except : # 중간에 비어있는 글자, na null 등 건너뛰기
#             print("scout 00 :", data[i])
            pass


    vectorizer = CountVectorizer(analyzer='word',       
                             min_df=min_df,   # 특정 횟수 이상 언급된 것만 따로                      
                             max_features=50000,             
                            )

    data_vectorized = vectorizer.fit_transform(data_morph)


    NUM_TOPICS = NUM_TOPICS # 분류하는 카테고리 숫자
    lda_model = LatentDirichletAllocation(
                            n_components=NUM_TOPICS,              
                            max_iter=max_iteration,               
                            learning_method='online',   
                            random_state=100,          
                            batch_size=128,            
                            evaluate_every = -1,       
                            #n_jobs = -1,               
                        )
    lda_output = lda_model.fit_transform(data_vectorized)
    log = lda_model.score(data_vectorized)
    per = lda_model.perplexity(data_vectorized)
    
    # Create Document - Topic Matrix
    lda_output = lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Styling
    def color_green(val):
        color = 'green' if val > .1 else 'black'
        return 'color: {col}'.format(col=color)

    def make_bold(val):
        weight = 700 if val > .1 else 400
        return 'font-weight: {weight}'.format(weight=weight)

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    
    # Create Document - Topic Matrix
    lda_output = lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Styling
    def color_green(val):
        color = 'green' if val > .1 else 'black'
        return 'color: {col}'.format(col=color)

    def make_bold(val):
        weight = 700 if val > .1 else 400
        return 'font-weight: {weight}'.format(weight=weight)

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    print(df_document_topics)
    
    name_02 = name + "_reference" + ".html"
        
    pyLDAvis.enable_notebook()
    p = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
    
    pyLDAvis.save_html(p,name_02)
    
        # Show top n keywords for each topic
    def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
        keywords = np.array(vectorizer.get_feature_names())
        topic_keywords = []
        for topic_weights in lda_model.components_:
            top_keyword_locs = (-topic_weights).argsort()[:n_words]
            topic_keywords.append(keywords.take(top_keyword_locs))
        return topic_keywords

    topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=11) 
    
    name_03 = name + "_reference" +".csv"
    pd.DataFrame(topic_keywords).to_csv(name_03, index=False, encoding='utf-8-sig')
    print(pd.DataFrame(topic_keywords));

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


    0    1    2    3    4   5    6   7    8   9    10
0   출마   선언  윤석열  이재명   대선  지사  민주당  공식   대해  주자   조국
1   의원  윤석열   총장   출마   국민  참석   회견  선언   요약  거나  윤봉길
2  윤석열   출마   선언   대선  노동자  민주  김건희  검증  이낙연  다음   노총


In [20]:
###### pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.114841  0.056216       1        1  48.003991
2      0.004388 -0.106445       2        1  35.396568
1     -0.119228  0.050229       3        1  16.599442, topic_info=    Term       Freq      Total Category  logprob  loglift
54    의원   8.000000   8.000000  Default  30.0000  30.0000
72    총장  10.000000  10.000000  Default  29.0000  29.0000
7     국민   7.000000   7.000000  Default  28.0000  28.0000
57   이재명  25.000000  25.000000  Default  27.0000  27.0000
76    회견   4.000000   4.000000  Default  26.0000  26.0000
..   ...        ...        ...      ...      ...      ...
38    선언   3.815867  39.522508   Topic3  -3.2054  -0.5419
25   문재인   0.498741   5.338116   Topic3  -5.2402  -0.5747
2   검찰총장   0.702943   8.834101   Topic3  -4.8970  -0.7353
62    정부   0.478142   6.102816   Topic3  -5.2824  -0.7508
17    대선   0.661258  27.944567   Topic3  -4.9582  -1.9480

[134 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
0         2  0.225698    거나
0         3  0.677094    거나
1         2  0.667149    검증
1         3  0.266860    검증
2         1  0.565989  검찰총장
...     ...       ...   ...
73        2  0.334282    출마
73        3  0.092856    출마
74        2  0.810845   테마주
75        1  0.789167    통한
76        3  0.882930    회견

[128 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])

In [36]:
for name in list :
    name_01 = name + ".csv"
    keyword = "이재명"
    
    df = pd.read_csv(name_01).dropna()
    df2 = df[df['keyword'] == keyword]
    # Convert to list
    data = df2.cleanBody.values.tolist()

    # user text에서 명사만 추출 

    from konlpy.tag import *
    from nltk import Text

    okt = Okt()

    data_morph = []
    for i in range(len(data)):
        data_morph.append(" ".join(okt.nouns(data[i])))

    vectorizer = CountVectorizer(analyzer='word',       
                             min_df=min_df,   # 특정 횟수 이상 언급된 것만 따로                      
                             max_features=50000,             
                            )

    data_vectorized = vectorizer.fit_transform(data_morph)


    NUM_TOPICS = NUM_TOPICS # 분류하는 카테고리 숫자
    lda_model = LatentDirichletAllocation(
                            n_components=NUM_TOPICS,              
                            max_iter=max_iteration,               
                            learning_method='online',   
                            random_state=100,          
                            batch_size=128,            
                            evaluate_every = -1,       
                            #n_jobs = -1,               
                        )
    lda_output = lda_model.fit_transform(data_vectorized)
    log = lda_model.score(data_vectorized)
    per = lda_model.perplexity(data_vectorized)
    
    # Create Document - Topic Matrix
    lda_output = lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Styling
    def color_green(val):
        color = 'green' if val > .1 else 'black'
        return 'color: {col}'.format(col=color)

    def make_bold(val):
        weight = 700 if val > .1 else 400
        return 'font-weight: {weight}'.format(weight=weight)

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    
    # Create Document - Topic Matrix
    lda_output = lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Styling
    def color_green(val):
        color = 'green' if val > .1 else 'black'
        return 'color: {col}'.format(col=color)

    def make_bold(val):
        weight = 700 if val > .1 else 400
        return 'font-weight: {weight}'.format(weight=weight)

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    print(df_document_topics)
    
    name_02 = keyword + "_reference" + ".html"
        
    pyLDAvis.enable_notebook()
    p = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
    
    pyLDAvis.save_html(p,name_02)
    
        # Show top n keywords for each topic
    def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
        keywords = np.array(vectorizer.get_feature_names())
        topic_keywords = []
        for topic_weights in lda_model.components_:
            top_keyword_locs = (-topic_weights).argsort()[:n_words]
            topic_keywords.append(keywords.take(top_keyword_locs))
        return topic_keywords

    topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=11) 
    
    name_03 = keyword +  "_reference" +".csv"
    pd.DataFrame(topic_keywords).to_csv(name_03, index=False, encoding='utf-8-sig')
    print(pd.DataFrame(topic_keywords));

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


    0    1    2    3   4    5    6   7    8    9    10
0  민주당   경선  지지자   진짜  정말   지금  이낙연  자기   검찰   하나   소리
1   사람   생각   정치   국민  문제  이재명   지지  이번   사면   중도   정도
2  이재명  대통령   대선  민주당  지사  윤석열   후보  의원  경기도  문재인  지지율


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.186777  0.236829       1        1  46.943508
1     -0.141504 -0.259652       2        1  28.966295
0      0.328281  0.022823       3        1  24.090197, topic_info=   Term        Freq       Total Category  logprob  loglift
22   사람  263.000000  263.000000  Default  30.0000  30.0000
34  이재명  651.000000  651.000000  Default  29.0000  29.0000
25   생각  130.000000  130.000000  Default  28.0000  28.0000
3    경선  106.000000  106.000000  Default  27.0000  27.0000
10  대통령  167.000000  167.000000  Default  26.0000  26.0000
..  ...         ...         ...      ...      ...      ...
18   민주    0.357357   33.251341   Topic3  -8.0036  -3.1097
4    국민    0.839327   84.116146   Topic3  -7.1497  -3.1840
34  이재명    5.257658  651.603207   Topic3  -5.3149  -3.3964
37   자신    0.392195   38.311121   Topic3  -7.9106  -3.1584
28   언론    0.417040   44.194831   Topic3  -7.8492  -3.2398

[127 rows x 6 columns], token_table=      Topic      Freq Term
term                      
0         3  0.987081   검찰
1         1  0.976854   결과
2         1  0.992242  경기도
3         1  0.018861   경선
3         2  0.009431   경선
...     ...       ...  ...
52        1  0.021335   하나
52        3  0.981430   하나
53        2  0.975381   해도
54        1  0.973928   후보
54        2  0.031083   후보

[89 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

In [37]:
for name in list :
    name_01 = name + ".csv"
    keyword = "이낙연"
    
    df = pd.read_csv(name_01).dropna()
    df2 = df[df['keyword'] == keyword]
    # Convert to list
    data = df2.cleanBody.values.tolist()

    # user text에서 명사만 추출 

    from konlpy.tag import *
    from nltk import Text

    okt = Okt()

    data_morph = []
    for i in range(len(data)):
        data_morph.append(" ".join(okt.nouns(data[i])))

    vectorizer = CountVectorizer(analyzer='word',       
                             min_df=min_df,   # 특정 횟수 이상 언급된 것만 따로                      
                             max_features=50000,             
                            )

    data_vectorized = vectorizer.fit_transform(data_morph)


    NUM_TOPICS = NUM_TOPICS # 분류하는 카테고리 숫자
    lda_model = LatentDirichletAllocation(
                            n_components=NUM_TOPICS,              
                            max_iter=max_iteration,               
                            learning_method='online',   
                            random_state=100,          
                            batch_size=128,            
                            evaluate_every = -1,       
                            #n_jobs = -1,               
                        )
    lda_output = lda_model.fit_transform(data_vectorized)
    log = lda_model.score(data_vectorized)
    per = lda_model.perplexity(data_vectorized)
    
    # Create Document - Topic Matrix
    lda_output = lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Styling
    def color_green(val):
        color = 'green' if val > .1 else 'black'
        return 'color: {col}'.format(col=color)

    def make_bold(val):
        weight = 700 if val > .1 else 400
        return 'font-weight: {weight}'.format(weight=weight)

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    
    # Create Document - Topic Matrix
    lda_output = lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Styling
    def color_green(val):
        color = 'green' if val > .1 else 'black'
        return 'color: {col}'.format(col=color)

    def make_bold(val):
        weight = 700 if val > .1 else 400
        return 'font-weight: {weight}'.format(weight=weight)

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    print(df_document_topics)
    
    name_02 = keyword + "_reference" + ".html"
        
    pyLDAvis.enable_notebook()
    p = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
    
    pyLDAvis.save_html(p,name_02)
    
        # Show top n keywords for each topic
    def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
        keywords = np.array(vectorizer.get_feature_names())
        topic_keywords = []
        for topic_weights in lda_model.components_:
            top_keyword_locs = (-topic_weights).argsort()[:n_words]
            topic_keywords.append(keywords.take(top_keyword_locs))
        return topic_keywords

    topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=11) 
    
    name_03 = keyword +  "_reference" +".csv"
    pd.DataFrame(topic_keywords).to_csv(name_03, index=False, encoding='utf-8-sig')
    print(pd.DataFrame(topic_keywords));

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


    0    1    2   3    4   5    6    7    8   9    10
0  민주당  이재명  이낙연  그냥   대선  이제   하나   자기   사과  대표  지지율
1  이낙연   생각   사면  지금  지지율  정도   진짜   본인  대통령  자기   그냥
2   사람   대표   사과  국민   정치  자기  대통령  민주당  이낙연  정도   그냥


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.192248  0.255609       1        1  35.364602
2     -0.357522  0.013188       2        1  32.473094
0      0.165274 -0.268797       3        1  32.162304, topic_info=   Term        Freq       Total Category  logprob  loglift
8    사람  127.000000  127.000000  Default  21.0000  21.0000
12  이재명  107.000000  107.000000  Default  20.0000  20.0000
4    대표  104.000000  104.000000  Default  19.0000  19.0000
5   민주당  119.000000  119.000000  Default  18.0000  18.0000
7    사과  100.000000  100.000000  Default  17.0000  17.0000
..  ...         ...         ...      ...      ...      ...
7    사과    0.620700  100.020211   Topic3  -6.6417  -3.9479
4    대표    0.601975  104.247539   Topic3  -6.6723  -4.0199
10   생각    0.490412  100.030386   Topic3  -6.8773  -4.1836
9    사면    0.352601   72.378148   Topic3  -7.2072  -4.1899
8    사람    0.421036  127.332624   Topic3  -7.0298  -4.5775

[84 rows x 6 columns], token_table=      Topic      Freq Term
term                      
0         2  0.991780   국민
1         1  0.020937   그냥
1         2  0.020937   그냥
1         3  0.963124   그냥
2         3  0.987106   대선
3         1  0.662031  대통령
3         2  0.331016  대통령
4         2  0.988033   대표
4         3  0.009593   대표
5         2  0.050133  민주당
5         3  0.944164  민주당
6         1  0.963319   본인
7         2  0.989800   사과
7         3  0.009998   사과
8         2  0.997388   사람
9         1  0.994775   사면
10        1  0.989699   생각
10        2  0.009997   생각
11        1  0.594916  이낙연
11        2  0.016680  이낙연
11        3  0.389197  이낙연
12        3  0.994605  이재명
13        1  0.023690   이제
13        3  0.971288   이제
14        1  0.079806   자기
14        2  0.678353   자기
14        3  0.239419   자기
15        1  0.971567   정도
15        2  0.025568   정도
16        2  0.988876   정치
17        1  0.976463   지금
18        1  0.982268  지지율
18        3  0.022324  지지율
19        1  0.982568   진짜
20        3  0.973330   하나, R=21, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

In [38]:
for name in list :
    name_01 = name + ".csv"
    keyword = "정세균"
    
    df = pd.read_csv(name_01).dropna()
    df2 = df[df['keyword'] == keyword]
    # Convert to list
    data = df2.cleanBody.values.tolist()

    # user text에서 명사만 추출 

    from konlpy.tag import *
    from nltk import Text

    okt = Okt()

    data_morph = []
    for i in range(len(data)):
        data_morph.append(" ".join(okt.nouns(data[i])))

    vectorizer = CountVectorizer(analyzer='word',       
                             min_df=min_df,   # 특정 횟수 이상 언급된 것만 따로                      
                             max_features=50000,             
                            )

    data_vectorized = vectorizer.fit_transform(data_morph)


    NUM_TOPICS = NUM_TOPICS # 분류하는 카테고리 숫자
    lda_model = LatentDirichletAllocation(
                            n_components=NUM_TOPICS,              
                            max_iter=max_iteration,               
                            learning_method='online',   
                            random_state=100,          
                            batch_size=128,            
                            evaluate_every = -1,       
                            #n_jobs = -1,               
                        )
    lda_output = lda_model.fit_transform(data_vectorized)
    log = lda_model.score(data_vectorized)
    per = lda_model.perplexity(data_vectorized)
    
    # Create Document - Topic Matrix
    lda_output = lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Styling
    def color_green(val):
        color = 'green' if val > .1 else 'black'
        return 'color: {col}'.format(col=color)

    def make_bold(val):
        weight = 700 if val > .1 else 400
        return 'font-weight: {weight}'.format(weight=weight)

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    
    # Create Document - Topic Matrix
    lda_output = lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Styling
    def color_green(val):
        color = 'green' if val > .1 else 'black'
        return 'color: {col}'.format(col=color)

    def make_bold(val):
        weight = 700 if val > .1 else 400
        return 'font-weight: {weight}'.format(weight=weight)

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    print(df_document_topics)
    
    name_02 = keyword + "_reference" + ".html"
        
    pyLDAvis.enable_notebook()
    p = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
    
    pyLDAvis.save_html(p,name_02)
    
        # Show top n keywords for each topic
    def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
        keywords = np.array(vectorizer.get_feature_names())
        topic_keywords = []
        for topic_weights in lda_model.components_:
            top_keyword_locs = (-topic_weights).argsort()[:n_words]
            topic_keywords.append(keywords.take(top_keyword_locs))
        return topic_keywords

    topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=11) 
    
    name_03 = keyword +  "_reference" +".csv"
    pd.DataFrame(topic_keywords).to_csv(name_03, index=False, encoding='utf-8-sig')
    print(pd.DataFrame(topic_keywords));

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


    0   1   2
0  대선  총리  검찰
1  검찰  총리  대선
2  총리  대선  검찰


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.256826 -0.097305       1        1  50.297883
0      0.347426 -0.036926       2        1  27.436607
2     -0.090601  0.134231       3        1  22.265510, topic_info=  Term       Freq      Total Category  logprob  loglift
1   대선  45.000000  45.000000  Default   3.0000   3.0000
2   총리  55.000000  55.000000  Default   2.0000   2.0000
0   검찰  53.000000  53.000000  Default   1.0000   1.0000
0   검찰  52.626073  53.787938   Topic1  -0.3930   0.6654
2   총리  24.710822  55.839608   Topic1  -1.1490  -0.1280
1   대선   0.624824  45.372454   Topic1  -4.8265  -3.5980
1   대선  41.288036  45.372454   Topic2  -0.0296   1.1990
2   총리   0.823114  55.839608   Topic2  -3.9448  -2.9239
0   검찰   0.415590  53.787938   Topic2  -4.6282  -3.5698
2   총리  30.305672  55.839608   Topic3  -0.1300   0.8910
1   대선   3.459594  45.372454   Topic3  -2.3001  -1.0716
0   검찰   0.746275  53.787938   Topic3  -3.8340  -2.7756, token_table=      Topic      Freq Term
term                      
0         1  0.985351   검찰
0         3  0.018592   검찰
1         1  0.022040   대선
1         2  0.903632   대선
1         3  0.066119   대선
2         1  0.447711   총리
2         2  0.017908   총리
2         3  0.537253   총리, R=3, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])